# Setups, Installations and Imports

In [1]:
%%capture 
!pip install tensorflow
!pip install tensorflow-addons
!pip install bayesian-optimization

In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import resnet50

import tensorflow_addons as tfa
import sys

In [3]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    IN_COLAB = True
except:
    IN_COLAB = False

In [4]:
import os
os.environ["TF_DETERMINISTIC_OPS"] = "1"
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

import io
import itertools
from functools import partial
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, recall_score
from tqdm.notebook import tqdm_notebook
from ipywidgets import IntProgress

from sklearn.model_selection import KFold, StratifiedKFold

from bayes_opt import BayesianOptimization

from sklearn.preprocessing import LabelBinarizer


# Download and Prepare Dataset

In [5]:
# Insert the directoryimport sys
if IN_COLAB:
    '''this is the exact path of the mounted direttory. in order to set it, right click on colab notebooks folder and copy path, and insert below '''
    COLAB_NOTEBOOKES_PATH = "/content/drive/MyDrive/Colab Notebooks"
    sys.path.insert(0,COLAB_NOTEBOOKES_PATH)  

import Ensembles_prepare_datasets


The syntax of the command is incorrect.
'touch' is not recognized as an internal or external command,
operable program or batch file.
The system cannot find the path specified.
'cat' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.


#### Dataloader

In [6]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128

def preprocess_image(image, label):
  img = tf.cast(image, tf.float32)
  img = img/255.

  return img, label


def get_loaders(x_train, y_train, x_test, y_test):
    trainloader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    testloader = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    trainloader = (
        trainloader
        .shuffle(1024)
        .map(preprocess_image, num_parallel_calls=AUTO)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )

    testloader = (
        testloader
        .map(preprocess_image, num_parallel_calls=AUTO)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )
    return trainloader, testloader

In [7]:
ROOT_PATH = './'
## SmallCNN models with different initialization
models_dir = 'IndependentSolutions'
MODEL_PATH = ROOT_PATH+'IndependentSolutions/'

if models_dir not in os.listdir(ROOT_PATH):
    os.mkdir(models_dir)

# Model

In [8]:
def Model1(num_labels, img_shape, color = 3):
  inputs = keras.layers.Input(shape=(img_shape, img_shape, color))

  x = keras.layers.Conv2D(16, (3,3), padding='same')(inputs)
  x = keras.activations.relu(x)
  x = keras.layers.MaxPooling2D(2, strides=2)(x)

  x = keras.layers.Conv2D(32,(3,3), padding='same')(x)
  x = keras.activations.relu(x)
  x = keras.layers.MaxPooling2D(2, strides=2)(x)

  x = keras.layers.Conv2D(32,(3,3), padding='same')(x)
  x = keras.activations.relu(x)
  x = keras.layers.MaxPooling2D(2, strides=2)(x)

  x = keras.layers.GlobalAveragePooling2D()(x)
  x = keras.layers.Dense(32, activation='relu')(x)
  x = keras.layers.Dropout(0.1)(x)
  
  outputs = keras.layers.Dense(num_labels, activation='softmax')(x)

  return keras.models.Model(inputs=inputs, outputs=outputs)

def Model():
    if color:
        return Model1(num_labels, img_shape,color)
    else:
        return Model1(num_labels, img_shape)

# Init hyper parameters

In [9]:

learning_rate_range = [0.001,0.1]
epsilon_range = [1e-08,1e-05]


# Callbacks

#### LR Scheduler

In [10]:

def lr_schedule(epoch,lr):
    if (epoch >= 0) & (epoch < 9):
        return lr
    elif (epoch >= 9) & (epoch < 19):
        return lr/2
    elif (epoch >= 19) & (epoch < 29):
        return lr/4
    else:
        return lr/8

#lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda epoch: lr_schedule(epoch), verbose=True)
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

# Independent Solutions

In [11]:
# Saving 5 independent solutions i.e, 5 different inits.
# Will save last model snapshot. 

def run_independent_solutions():

    EPOCHS = 10
    SAVE_PATH = 'IndependentSolutions/'

    for iter in tqdm_notebook(range(5)):
      # initialize weights
      keras.backend.clear_session()
      model = Model()
      #optimizer define with name
      opt = tf.keras.optimizers.Adam(learning_rate=0.001, epsilon=1e-07, name='Adam'+str(iter))
      # compile
      model.compile(opt, 'sparse_categorical_crossentropy', metrics=['accuracy'])
      # train
      _ = model.fit(trainloader,
                  epochs=EPOCHS,
                  validation_data=testloader,
                  callbacks=[lr_callback],
                  verbose=0)
      # save final model snapshot
      model.save(SAVE_PATH+'smallcnn_independent_model_{}.h5'.format(iter))
      # evaluate
      loss, accuracy = model.evaluate(testloader, verbose=0)
      print("Test Error Rate: ", round((1-accuracy)*100, 2), '% | Iteration No: ', iter)

def run_independent_solutions(lr,ep,trainloader,testloader):

    EPOCHS = 10
    SAVE_PATH = 'IndependentSolutions/'

    for iter in tqdm_notebook(range(1)):
      # initialize weights
      keras.backend.clear_session()
      model = Model()
      #optimizer define with name
      opt = tf.keras.optimizers.Adam(learning_rate=lr, epsilon=ep, name='Adam'+str(iter))
      # compile
      model.compile(opt, 'sparse_categorical_crossentropy', metrics=['accuracy'])
      # train
      _ = model.fit(trainloader,
                  epochs=EPOCHS,
                  validation_data=testloader,
                  callbacks=[lr_callback],
                  verbose=0)
      # save final model snapshot
      model.save(SAVE_PATH+'smallcnn_independent_model_{}.h5'.format(iter))
      # evaluate
      loss, accuracy = model.evaluate(testloader, verbose=0)
      print("Test Error Rate: ", round((1-accuracy)*100, 2), '% | Iteration No: ', iter)
#run_independent_solutions()

# Ensembles


# Deep Ensembles

In [12]:
# make an ensemble prediction for multi-class classification
def ensemble_predictions(members):
	# make predictions
	yhats = [model.predict(x_test/255.) for model in members]
	yhats = np.array(yhats)
	# sum across ensemble members
	summed = np.sum(yhats, axis=0)
	# argmax across classes
	result = np.argmax(summed, axis=1)
	return result
 
# evaluate a specific number of members in an ensemble
def evaluate_n_members(members):
    accuracy_list = []
    for n_members in tqdm_notebook(range(1, len(members))):
        # select a subset of members
        subset = members[:n_members]
        print(len(subset))
        # make prediction
        yhat = ensemble_predictions(subset)
        # calculate accuracy
        accuracy_list.append(100*accuracy_score(y_test, yhat))

    return accuracy_list

def ensemble_predictions(members, test):
	# make predictions
	yhats = [model.predict(test/255.) for model in members]
	yhats = np.array(yhats)
	# sum across ensemble members
	summed = np.sum(yhats, axis=0)
	# argmax across classes
	result = np.argmax(summed, axis=1)
	return result

In [13]:
# Run the ensembles for plot
def run_ensembles_and_plot():
    members = [tf.keras.models.load_model(MODEL_PATH + model_ckpts[i]) 
                            for i in range(len(model_ckpts))]
    
    single_model = np.random.choice(members)
    accuracy_single_model = single_model.evaluate(testloader)[1] * 100
    
    accuracy_list = evaluate_n_members(members)
    
    accuracy_list.insert(0, accuracy_single_model)
    rng = [i for i in range(0, len(members))]
    
    plt.figure(figsize=(9,8))
    plt.plot(rng, accuracy_list, label='deep ensemble')
    plt.plot(rng, [accuracy_single_model]*len(rng), '--', label='single model')
    plt.title("Test accuracy as a function of ensemble size")
    plt.xlabel("Ensemble size")
    plt.ylabel("Test accuracy")
    plt.legend();
    plt.savefig('ensemble_func.png')
    
#run_ensembles_and_plot()

In [14]:
# Bayesian Optimization with k fold validation 
def iner_kfold(x_train, y_train, lr, ep):
    kf = KFold(n_splits = 3, random_state=3, shuffle=True)
    accuracy_list = []
    for train_index, test_index in kf.split(x_train):
        f_x_train, f_x_test = x_train[train_index], x_train[test_index]
        f_y_train, f_y_test = y_train[train_index], y_train[test_index]
        trainloader, testloader = get_loaders(f_x_train, f_y_train, f_x_test, f_y_test)
        run_independent_solutions(lr, ep,trainloader,testloader)
        model_ckpts = os.listdir(MODEL_PATH)
        members = [tf.keras.models.load_model(MODEL_PATH + model_ckpts[i]) 
                            for i in range(len(model_ckpts))]
        yhat = ensemble_predictions(members,f_x_test)
        accuracy = 100*accuracy_score(f_y_test, yhat)
        accuracy_list.append(accuracy)
    print('accuracy:', np.mean(accuracy_list), '+/-', np.std(accuracy_list))
    return np.mean(accuracy_list)

def kfold_opt(x_train, y_train):
    print("finding best hyper parameters")
    fit_with_partial = partial(iner_kfold, x_train, y_train)
    pbounds = { 'lr': (0.0001,0.01), 'ep': (1e-8,1e-5)}

    optimizer = BayesianOptimization(
        f=fit_with_partial,
        pbounds=pbounds,
        verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
        random_state=1,
    )

    optimizer.maximize(init_points=10, n_iter=10)


    for i, res in enumerate(optimizer.res):
        print("Iteration {}: \n\t{}".format(i, res))

    print(optimizer.max)
    return optimizer.max['params']['lr'], optimizer.max['params']['ep']




In [15]:
#claculating the result TPR and FPR
def tpr_fpr(y_true, y_prediction):

    cnf_matrix = confusion_matrix(y_true, y_prediction)
   
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float).sum()
    FN = FN.astype(float).sum()
    TP = TP.astype(float).sum()
    TN = TN.astype(float).sum()

    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)
    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)
    return TPR, FPR

In [16]:
#writing the results to a file
def write_res(name, algo, cros_val, hp_lr, hp_ep, accu, tpr, fpr, preci, auc, train_time):
    file_name = name + '-' + algo + ".csv"
    print(file_name)
    f = open(file_name, "a")
    f.write(name+','+algo+','+str(cros_val)+','+str(hp_lr)+','+str(hp_ep)+','
            +str(accu)+','+str(tpr)+','+str(fpr)+','+str(preci)+','+str(auc)+','+str(train_time)+'\n')
    f.close()

In [17]:
#get AUC of multiclass results
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average=average)

In [18]:
# validate preformence result with kfold 
def kfold_val():
    f = open(n + "-smallcnn.csv", "w")
    f.write('name, algo, cros_val, hp_lr, hp_ep, accu, tpr, fpr, preci, auc, train_time\n')
    f.close()
    kf = KFold(n_splits=10, random_state=3, shuffle=True) #should be 10
    start1 = time.time()
    lr=0.001
    ep=1e-07
    accuracy_list = []
    training_times = []
    first_fold = True
    cros_iter = 1
    for train_index, test_index in kf.split(x):
        print("TRAIN:", train_index, "TEST:", test_index)
        f_x_train, f_x_test = x[train_index], x[test_index]
        f_y_train, f_y_test = y[train_index], y[test_index]
        start2 = time.time()
        if first_fold:
            #find best hyperparameters to use
            lr, ep = kfold_opt(f_x_train, f_y_train)
            first_fold = False
        
        trainloader, testloader = get_loaders(f_x_train, f_y_train, f_x_test, f_y_test) 
        run_independent_solutions(lr,ep,trainloader,testloader)
        end2 = time.time()
        model_ckpts = os.listdir(MODEL_PATH)
        members = [tf.keras.models.load_model(MODEL_PATH + model_ckpts[i]) 
                            for i in range(len(model_ckpts))]
        yhat = ensemble_predictions(members,f_x_test)
        ##### infer results and saving
        accuracy = 100*accuracy_score(f_y_test, yhat)
        TPR, FPR = tpr_fpr(f_y_test, yhat)
        precision = precision_score(f_y_test, yhat, average='weighted')
        auc = multiclass_roc_auc_score(f_y_test,yhat)
        accuracy_list.append(accuracy)
        train_time = end2 - start2
        training_times.append(train_time)
        print("accuracy: {}".format(accuracy))
        write_res(n, 'smallcnn', cros_iter, lr, ep, accuracy, TPR, FPR, precision, auc, train_time)
        cros_iter = cros_iter + 1
    end1 = time.time()
    print('accuracy:', np.mean(accuracy_list), '+/-', np.std(accuracy_list))
    print("avarage accuracy: {}".format(np.average(accuracy_list)) )   
    print("Network takes {:.3f} minuts to fold validate".format((end1 - start1)/60))

#kfold_val()

In [ ]:
#model_list = ['cifar10', 'cifar100', 'mnist', 'flowers102', 'monkeys10', 'intel','cifar10_2', 'cifar100_2', 'mnist_2', 'flowers102_2', 'monkeys10_2', 'intel_2']

#model_list = ['cifar100', 'cifar100_2', 'flowers102', 'flowers102_2']
model_list = ['cifar10_2', 'cifar100', 'cifar100_2', 'mnist', 'mnist_2']
color = 0
for n in model_list:
    if n == 'cifar10':
        x,x2,y,y2,num_labels,img_shape = Ensembles_prepare_datasets.load_cifar10()
    elif n == 'cifar10_2':
        x1,x,y1,y,num_labels,img_shape = Ensembles_prepare_datasets.load_cifar10()
    elif n == 'cifar100':
        x,x2,y,y2,num_labels,img_shape = Ensembles_prepare_datasets.load_cifar100()
    elif n == 'cifar100_2':
        x1,x,y1,y,num_labels,img_shape = Ensembles_prepare_datasets.load_cifar100()
    elif n == 'mnist':
        x,x2,y,y2,num_labels,img_shape,color = Ensembles_prepare_datasets.load_mnist()
    elif n == 'mnist_2':
        x1,x,y1,y,num_labels,img_shape,color = Ensembles_prepare_datasets.load_mnist()
    elif n == 'flowers102':
        x,x2,y,y2,num_labels,img_shape = Ensembles_prepare_datasets.load_flowers102()
    elif n == 'flowers102_2':
        x1,x,y1,y,num_labels,img_shape = Ensembles_prepare_datasets.load_flowers102()
        
    
    elif n == 'monkeys10':
        x,x2,y,y2,num_labels,img_shape = Ensembles_prepare_datasets.load_monkeys10()
    elif n == 'monkeys10_2':
        x1,x,y1,y,num_labels,img_shape = Ensembles_prepare_datasets.load_monkeys10()
    elif n == 'intel':
        x,x2,y,y2,num_labels,img_shape = Ensembles_prepare_datasets.load_intel()
    elif n == 'intel_2':
        x1,x,y1,y,num_labels,img_shape = Ensembles_prepare_datasets.load_intel()
    kfold_val()

TRAIN: [    0     1     2 ... 29996 29998 29999] TEST: [    5    33    37 ... 29967 29978 29997]
finding best hyper parameters
|   iter    |  target   |    ep     |    lr     |
-------------------------------------------------


  0%|          | 0/1 [00:00<?, ?it/s]

Test Error Rate:  51.01 % | Iteration No:  0


  0%|          | 0/1 [00:00<?, ?it/s]

Test Error Rate:  51.03 % | Iteration No:  0


...

  0%|          | 0/1 [00:00<?, ?it/s]

Test Error Rate:  63.7 % | Iteration No:  0
accuracy: 36.3
cifar100_2-smallcnn.csv
TRAIN: [    0     1     5 ... 29997 29998 29999] TEST: [    2     3     4 ... 29992 29995 29996]


  0%|          | 0/1 [00:00<?, ?it/s]

Test Error Rate:  65.83 % | Iteration No:  0
accuracy: 34.166666666666664
cifar100_2-smallcnn.csv
TRAIN: [    0     1     2 ... 29995 29996 29997] TEST: [   12    32    44 ... 29993 29998 29999]


  0%|          | 0/1 [00:00<?, ?it/s]